In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 969 kB/s 
     |████████████████████████████████| 3.1 MB 1.7 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=ed8a6856f392a536a0fab67c3b6515c53a90af82fad0cb8fb465e8009d6dd961
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [2]:
import pandas as pd
import collections
import random 
from nltk.cluster import kmeans, cosine_distance, euclidean_distance
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1', device='cuda')
n_clusters = 10

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [3]:
df = pd.read_csv('../input/proc-df/proc_df.csv', dtype=str)
df = df.copy()
df["proc_res2"] = df["proc_res"]
df = df.groupby('proc_res2').first().set_index('Unnamed: 0')
df

,proc_og,proc_res,raw_og,raw_res,nostop_og,nostop_res,og_id,res_id
Unnamed: 0,,,,,,,,
24542,es la llave de la para volver a gobernar CABA .,! ? . Voté a Balbín por 2 en el 73 ( allí por...,@spitta1969 @GugaLusto es la llave de la @UCRC...,@bbponti @spitta1969 @GugaLusto @UCRCapital .....,llave volver a gobernar CABA .,! ? . Voté a Balbín 2 73 ( única a Rúa ) ; a ...,1354751894678351872,1354808644286570507
42979,"No sirve militar vacunas , sirve traerlas",! Ahh sos un viejo / a macrista aspirante a t...,"@norabar No sirve militar vacunas, sirve traerlas",@PizzaConMuza @norabar ! Ahh sos un viejo/a ma...,"sirve militar vacunas , sirve traerlas",! Ahh sos viejo / a macrista aspirante a troo...,1358142043571310592,1358166438868049926
32164,El tono del libro está fijado ya en la dedica...,"! Ahhhh , bue !",El tono del libro está fijado ya en la dedicat...,"@ernestorr @cachyibarra !!! Ahhhh, bue !!!!","tono libro fijado dedicatoria : "" incontables...","! Ahhhh , bue !",1368369235156738053,1368635128944197641
33321,"Todo el éxito en esta nueva etapa , . Confiam...",! En Peru renunciaron 400 responsables del va...,"Todo el éxito en esta nueva etapa, @carlavizzo...",@DrMaceira @carlavizzotti @msalnacion #CarlaVi...,"éxito etapa , . Confiamos capacidad y experie...",! Peru renunciaron 400 responsables vacuna ga...,1363255006292561921,1363474147066675201
18679,Este verano disfrutá y cuidate . a donde vaya...,! no nos manda agua .. como nos vamos a lavar...,‼️ Este verano disfrutá y cuidate.\n\n➡️ A don...,@msalnacion #MalenaNoTenemosAgua !!! @AySA_Ofi...,"verano disfrutá y cuidate . a vayas , seguí c...",! manda agua .. a lavar manos ? 2 semanas agua .,1345383637139779585,1345710407714017282
...,...,...,...,...,...,...,...,...
21715,Qué placer es siempre leerlo Ernesto ! Es tan...,🏽en todo de acuerdo,@ernestorr @caballobosterok Qué placer es siem...,@nadilian @ernestorr @caballobosterok 👏🏽👏🏽👏🏽👏🏽...,"placer leerlo Ernesto ! , coherente , ojalá p...",🏽en,1372321312547467265,1372322892353048577
33309,a Albistur no lo conozco !,🏿jajaja Porque no tiene vacas,@losbustingo @spitta1969 @CarolinaPiparo A Alb...,@Ramiro488 @spitta1969 @CarolinaPiparo ✌🏿✌🏿Jua...,a Albistur conozco !,🏿jajaja vacas,1359125500619350024,1359127259064500228
38064,Cuando pensas que lleguemos a eso acá??mayo ?...,🥸,@rquiroga777 Cuando pensas q lleguemos a eso a...,@LedZepp24 @rquiroga777 ⏰🥸 https://t.co/WfXLG7...,pensas lleguemos a acá??mayo ? optimistas ?,🥸,1351719495916728321,1351722828245196803


In [4]:
#Primer método, Token Count Vectorizer - KMeans
#cv=CountVectorizer() 
#word_count_vector=cv.fit_transform(df["nostop_res"].astype('str'))
#normed_matrix = normalize(word_count_vector, axis=1, norm='max')
#dict_space = VarianceThreshold(threshold=1e-5).fit_transform(normed_matrix)
#svd = TruncatedSVD(n_components=300)
#red_emb = svd.fit_transform(dict_space.toarray())
#clusterer = KMeans(n_clusters=n_clusters, random_state=0)
#clusterer.fit(X=red_emb)
#df["label1"] = clusterer.labels_

In [5]:
#segundo método, Token Count Vectorizer - Fast_clustering
#clusters = util.community_detection(red_emb, min_community_size=50, threshold=0.40)
#print(f"número de tweets clusterizados: {sum(len(clus) for clus in clusters)}")
#df["label2"] = 99

#for i, cluster in enumerate(clusters):
#    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
#    for sentence_id in cluster[0:20]:
#        print("\t", df["proc_res"][sentence_id])
#for i, cluster in enumerate(clusters):
#    for sentence_id in cluster:
#        df["label2"][sentence_id] = i     

In [6]:
#tercer metodo, SBERT - KMeans
embedings = model.encode(list(df["nostop_res"].astype('str')), batch_size=70, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)

clustering_model = KMeans(n_clusters=n_clusters)
clustering_model.fit(embedings.cpu())
cluster_assignment = clustering_model.labels_
clustered_sentences = [[] for i in range(n_clusters)]

for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(df["proc_res"][sentence_id])
df["label3"] = clustering_model.labels_

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    for tw in random.sample(cluster, 20):
        print(tw)
#Mejora,elegir solo los q estan cerca de un treshold. el resto sacarles la tag

Batches:   0%|          | 0/783 [00:00<?, ?it/s]

Cluster  1
 Gracias , Martu ! Saludos a apu , mis condolencias en este momento tan duro
 gracias , Ernesto !
 Genial ! Gracias !
 Gracias ¡
 Gracias Nora ! Sin lugar a dudas hay que controlar más a los que llegan de Brasil .
 Gracias Martín por tu trabajo
 Una genia Hebe . Saludos a ambos !
 Gracias como siempre , Nori .
 Excelente noticia ¡
 Si estas viva agradecé
 si ! ya lo llené . gracias !
 No puedo , pero mandale saludos a la familia .
 Gracias Omar querido , gracias
 Gracias por el estado saneado que dejaron
 que es eso ? Me podes decir ? Gracias
 ¡ Gracias !
 Ernesto , gracias por tu esperanzador mensaje , Dios nos ampare
 Gracias Nora . Veremos mañana o el lunes . Pero es alarmante para un sabado estos datos
 Genia total . Gracias !
 Lo es ! Lamentablemente .
Cluster  2
 Están tan enfermos que si fueran un equipo de fútbol , festejarian los corners;lees lo que escriben y terminas confirmando por que el país esta como esta y vivimos como vivimos , son mononeuronales .
 Claro , 

In [7]:
#cuarto método, SBERT - Fast_clustering
clusters = util.community_detection(embedings, min_community_size=100, threshold=0.40)

df["label4"] = 99
print(f"número de tweets clusterizados: {sum(len(clus) for clus in clusters)}")
for i, cluster in enumerate(clusters):
    print("\nCluster {}".format(i+1))
    for sentence_id in cluster[0:20]:
        print("\t", df["proc_res"][sentence_id])
        
for i, cluster in enumerate(clusters):
    for sentence_id in cluster:
        df["label4"][sentence_id] = i     

número de tweets clusterizados: 13074

Cluster 1
	  ! En Peru renunciaron 400 responsables del vacuna gate , aca 1 ! De Alberso para abajo todos Komplices
	  " 10 millones de personas vacunadas con dos dosis o 20 millones de personas con una sola " . " Es muy lógico pensar que se pueda vacunar a más gente con la primera dosis y diferir la segunda para cuando ya esté controlado el brote . " Impecable
	  " 31 casos en un par de millones de vacunas aplicadas " . No discuto eso : pregunto cuál hubiera sido la reacción de Pitta de haberse dado esos 31 casos con a GenericJab . Creo que cualquier persona honesta intelectualmente lo puede imaginar sin problema .
	  " 37000 vacunados Chile " en Google y listo
	  " Alberto criminal ! No deja que los médicos de CABA se inmunicen con una vacuna que funciona ! " Cosas así . Hay que se audaz .
	  " Apología de la migración " ? ! Robar vacunas no , pero migrar es delito ? !
	  " Compra de la vacuna " las banderas de la anticuarentena y el antivacuna 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [8]:
df.drop(columns=["raw_res", "raw_og"], inplace=True)

In [9]:
df.to_csv("clustered_df.csv")

In [10]:
df

,proc_og,proc_res,nostop_og,nostop_res,og_id,res_id,label3,label4
Unnamed: 0,,,,,,,,
24542,es la llave de la para volver a gobernar CABA .,! ? . Voté a Balbín por 2 en el 73 ( allí por...,llave volver a gobernar CABA .,! ? . Voté a Balbín 2 73 ( única a Rúa ) ; a ...,1354751894678351872,1354808644286570507,1,99
42979,"No sirve militar vacunas , sirve traerlas",! Ahh sos un viejo / a macrista aspirante a t...,"sirve militar vacunas , sirve traerlas",! Ahh sos viejo / a macrista aspirante a troo...,1358142043571310592,1358166438868049926,7,99
32164,El tono del libro está fijado ya en la dedica...,"! Ahhhh , bue !","tono libro fijado dedicatoria : "" incontables...","! Ahhhh , bue !",1368369235156738053,1368635128944197641,5,1
33321,"Todo el éxito en esta nueva etapa , . Confiam...",! En Peru renunciaron 400 responsables del va...,"éxito etapa , . Confiamos capacidad y experie...",! Peru renunciaron 400 responsables vacuna ga...,1363255006292561921,1363474147066675201,9,0
18679,Este verano disfrutá y cuidate . a donde vaya...,! no nos manda agua .. como nos vamos a lavar...,"verano disfrutá y cuidate . a vayas , seguí c...",! manda agua .. a lavar manos ? 2 semanas agua .,1345383637139779585,1345710407714017282,2,99
...,...,...,...,...,...,...,...,...
21715,Qué placer es siempre leerlo Ernesto ! Es tan...,🏽en todo de acuerdo,"placer leerlo Ernesto ! , coherente , ojalá p...",🏽en,1372321312547467265,1372322892353048577,4,1
33309,a Albistur no lo conozco !,🏿jajaja Porque no tiene vacas,a Albistur conozco !,🏿jajaja vacas,1359125500619350024,1359127259064500228,8,99
38064,Cuando pensas que lleguemos a eso acá??mayo ?...,🥸,pensas lleguemos a acá??mayo ? optimistas ?,🥸,1351719495916728321,1351722828245196803,4,1
